In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

In [18]:
data = pd.read_csv("AusnetLBRAUpdated.csv", encoding='latin1')

In [19]:
data.head()

,ID,date,day,month,year,company,firedist,latitude,longitute,fireseason,...,Year,F,G,T,Calc,Code Calc,Geog_Recode,G.Recode.,Calc.Recode.,Geog Classify.Final
0,1,01.01.12,1,1,2012,AusNet,NORTH EAST,-36.44927,146.432100,YES,...,2012,1,4.6,0.5,2.3,E. 1 to 2.99,LBRA,0.2,0.1,REFCL
1,2,01.01.12,1,1,2012,AusNet,EAST GIPPSLAND,-37.56000,149.754080,YES,...,2012,1,1.0,0.5,0.5,F. 0.3 to 0.99,LBRA,0.2,0.1,HBRA
2,3,01.01.12,1,1,2012,Powercor,NORTHERN COUNTRY,-36.24266,145.223201,YES,...,2012,1,0.2,0.5,0.1,G. 0.1 to 0.29,LBRA,0.2,0.1,LBRA
3,4,02.01.12,2,1,2012,Jemena,CENTRAL,-37.57916,144.728090,YES,...,2012,1,0.2,2.0,0.4,F. 0.3 to 0.99,LBRA,0.2,0.4,LBRA
4,5,02.01.12,2,1,2012,United,CENTRAL,-38.05155,145.116510,YES,...,2012,1,0.2,2.0,0.4,F. 0.3 to 0.99,LBRA,0.2,0.4,LBRA


In [20]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2527 entries, 0 to 2526
Data columns (total 29 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   2527 non-null   int64  
 1   date                 2527 non-null   object 
 2   day                  2527 non-null   int64  
 3   month                2527 non-null   int64  
 4   year                 2527 non-null   int64  
 5   company              2527 non-null   object 
 6   firedist             2527 non-null   object 
 7   latitude             2455 non-null   float64
 8   longitute            2455 non-null   float64
 9   fireseason           2527 non-null   object 
 10  peak1                2527 non-null   object 
 11  peak2                2527 non-null   object 
 12  hbra                 2521 non-null   object 
 13  tfb                  2527 non-null   int64  
 14  ranking              2446 non-null   object 
 15  ranking_P2           2446 non-null   o

# Initializing weights for f_factor, g_factor, t-factor

In [21]:
# F-factor weight allocation: ground-fire (1.0) or other (0.2)

ground=1.0
other=1.0

# G-factor weight allocation: LBRA (0.25), HBRA (1.0), REFCL (1.5), PRF (4.5) 

lbra=0.2
hbra=1.0
refcl=4.6
prf=19.8

# T-factor weight allocation: No forecast (0.1), Low-Moderate (0.2), high (0.5), very high (1.0), severe (2.0), extreme (3.0), code red (5.0)

noforecast=0.1
low2mod=0.2
high=0.5
veryhigh=1.0
severe=2.0
extreme=3.5
codered=5

In [22]:
# Converting variables into respective vectors
f_factor = np.array([ground, other])
g_factor = np.array([lbra, hbra, refcl, prf])
t_factor = np.array([noforecast, low2mod, high, veryhigh, severe, extreme, codered])

In [79]:
# Updated dataframe
num_rows_original = data.shape[0]
#random_weight = np.full((num_rows_original,1), 2)
columns = ['date','day','month','year','company','firedist','fireseason','peak1',
           'hbra','tfb','Geog Classify.Final','danger_rating','ground_fire']
drawData = pd.DataFrame(data = data, columns=columns)
for i in range(num_rows_original):
  drawData.loc[i,'weights'] = 2

# use this data to sample 365 days for 10000 year
roll_data = drawData.dropna(subset=['danger_rating','Geog Classify.Final']).copy()
num_rows = roll_data.shape[0]


In [78]:
roll_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2521 entries, 0 to 2526
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   date                 2521 non-null   object 
 1   day                  2521 non-null   int64  
 2   month                2521 non-null   int64  
 3   year                 2521 non-null   int64  
 4   company              2521 non-null   object 
 5   firedist             2521 non-null   object 
 6   fireseason           2521 non-null   object 
 7   peak1                2521 non-null   object 
 8   hbra                 2521 non-null   object 
 9   tfb                  2521 non-null   int64  
 10  Geog Classify.Final  2521 non-null   object 
 11  danger_rating        2521 non-null   object 
 12  ground_fire          2521 non-null   object 
 13  weights              2521 non-null   float64
dtypes: float64(1), int64(4), object(9)
memory usage: 295.4+ KB


In [80]:
  # Adding extra column for managing weights
modified_roll = roll_data.copy()
base_penalty = 25000

modified_roll['F_ground'] = 0
modified_roll['F_ground'] = 0
modified_roll['F_other'] = 0
modified_roll['G_lbra'] = 0
modified_roll['G_hbra'] = 0
modified_roll['G_refcl'] = 0
modified_roll['G_prf'] = 0
modified_roll['T_noforecast'] = 0
modified_roll['T_low2mod'] = 0
modified_roll['T_high'] = 0
modified_roll['T_veryhigh'] = 0
modified_roll['T_severe'] = 0
modified_roll['T_extreme'] = 0
modified_roll['T_codered'] = 0
modified_roll['F_weight'] = 0
modified_roll['G_weight'] = 0
modified_roll['T_weight'] = 0
modified_roll['Penalty'] = 0
modified_roll['Penalty_ausnet'] = 0
modified_roll['Penalty_citi'] = 0
modified_roll['Penalty_jem'] = 0
modified_roll['Penalty_pow'] = 0
modified_roll['Penalty_uni'] = 0    

In [81]:
modified_roll.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2521 entries, 0 to 2526
Data columns (total 36 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   date                 2521 non-null   object 
 1   day                  2521 non-null   int64  
 2   month                2521 non-null   int64  
 3   year                 2521 non-null   int64  
 4   company              2521 non-null   object 
 5   firedist             2521 non-null   object 
 6   fireseason           2521 non-null   object 
 7   peak1                2521 non-null   object 
 8   hbra                 2521 non-null   object 
 9   tfb                  2521 non-null   int64  
 10  Geog Classify.Final  2521 non-null   object 
 11  danger_rating        2521 non-null   object 
 12  ground_fire          2521 non-null   object 
 13  weights              2521 non-null   float64
 14  F_ground             2521 non-null   int64  
 15  F_other              2521 non-null   i

In [112]:
for i in range(1725):
  if roll_data['ground_fire'][i]== 'YES': modified_roll.loc[i,['F_ground']] = 1
  if roll_data['ground_fire'][i] == 'NO': modified_roll.loc[i,['F_other']] = 1 
  
  if roll_data['Geog Classify.Final'][i] == 'LBRA': modified_roll.loc[i,['G_lbra']] = 1 
  if roll_data['Geog Classify.Final'][i] == 'HBRA': modified_roll.loc[i,['G_hbra']] = 1 
  if roll_data['Geog Classify.Final'][i] == 'REFCL': modified_roll.loc[i,['G_refcl']] = 1
  if roll_data['Geog Classify.Final'][i] == 'PRF': modified_roll.loc[i,['G_prf']] = 1
  
  if roll_data['danger_rating'][i] == 'NO FORECAST': modified_roll.loc[i,['T_noforecast']] = 1
  if roll_data['danger_rating'][i] == 'Low-Moderate': modified_roll.loc[i,['T_low2mod']] = 1
  if roll_data['danger_rating'][i] == 'LOW-MODERATE': modified_roll.loc[i,['T_low2mod']] = 1
  if roll_data['danger_rating'][i] == 'High': modified_roll.loc[i,['T_high']] = 1 
  if roll_data['danger_rating'][i] == 'Very high': modified_roll.loc[i,['T_veryhigh']] = 1
  if roll_data['danger_rating'][i] == 'VERY HIGH': modified_roll.loc[i,['T_veryhigh']] = 1
  if roll_data['danger_rating'][i] == 'Severe': modified_roll.loc[i,['T_severe']] = 1 
  if roll_data['danger_rating'][i] == 'Extreme': modified_roll.loc[i,['T_extreme']] = 1
  
  modified_roll.loc[i,['F_weight']] = (modified_roll.loc[i,['F_ground']] * ground) + (modified_roll.loc[i,['F_other']] * other)
  modified_roll.loc[i,['G_weight']] = (modified_roll.loc[i,['G_lbra']] * lbra) + (modified_roll.loc[i,['G_hbra']] * hbra) + (modified_roll.loc[i,['G_refcl']] * refcl) + (modified_roll.loc[i,['G_prf']] * prf)
  modified_roll.loc[i,['T_weight']] = (modified_roll.loc[i,['T_noforecast']] * noforecast) + (modified_roll.loc[i,['T_low2mod']] * low2mod) + (modified_roll.loc[i,['T_high']] * high) + (modified_roll.loc[i,['T_veryhigh']] * veryhigh) + (modified_roll.loc[i,['T_severe']] * severe) + (modified_roll.loc[i,['T_extreme']] * extreme)
  modified_roll.loc[i,['Penalty']] = base_penalty * modified_roll.loc[i,['F_weight']] * modified_roll.loc[i,['G_weight']] * modified_roll.loc[i,['T_weight']] * modified_roll.loc[i,['weights']] 
 
  if roll_data['company'][i] == 'AusNet': modified_roll.loc[i,['Penalty_ausnet']] = modified_roll.loc[i,['Penalty']]
  if roll_data['company'][i] == 'CitiPower': modified_roll.loc[i,['Penalty_citi']] = modified_roll.loc[i,['Penalty']]
  if roll_data['company'][i] == 'Jemena': modified_roll.loc[i,['Penalty_jem']] = modified_roll.loc[i,['Penalty']]
  if roll_data['company'][i] == 'Powercor': modified_roll.loc[i,['Penalty_pow']] = modified_roll.loc[i,['Penalty']]
  if roll_data['company'][i] == 'United': modified_roll.loc[i,['Penalty_uni']] = modified_roll.loc[i,['Penalty']] 
  
  

In [111]:
def hot_year(temp_data):
  temp12 = temp_data[temp_data['year'] == 2012, ]
  penalty = temp12[:,31:35].sum


0    0
1    0
2    0
3    0
4    0
Name: Penalty, dtype: int64

In [91]:
modified_roll['F_ground'][2]

2